# Description of the scripts used in the PCA analyses: discovery, replication and mega samples

The aim of this notebook is to describe the paremeters used uring the PCA calculation and outlier removal in the white Caucasian individuals from the UKB that had exome data

The notebooks indicated in this analyses such as `GWAS_QC.ipynb,PCA.ipynb` are publicly available [here](https://github.com/cumc/bioworkflows/tree/master/GWAS)

# Global variables

In [ ]:
pca_sos=$USER_PATH/bioworkflows/GWAS/PCA.ipynb
gwasqc_sos=$USER_PATH/bioworkflows/GWAS/GWAS_QC.ipynb
numThreads=1
job_size=1

## 08-30-21 Run with QC'ed genotype array data

## Step 1. Select "European individuals" from genotype array data

In [ ]:
#Columbia's cluster
cwd=$UKBB_PATH/results/083021_PCA_results/europeans
#bfile with sample and variants QC from 083021 containing all of the samples
##here I used the bfile in which individuals with call rate >90% were retained
genoFile=~/UKBiobank/genotype_files_processed/090221_sample_variant_qc_final_callrate90/cache/UKB_genotypedatadownloaded083019.090221_sample_variant_qc_final_callrate90.filtered.extracted.bed
#To keep the samples of white individuals only
keep_samples=$UKBB_yale/pleiotropy_R01/ukb43978_OCT2020/dc2325_phenotypes/030821_ukb42495_exomed_white_189010ind
#QC is already done, so no need to filter any more
maf_filter=0
geno_filter=0
hwe_filter=0
mind_filter=0
mem='30G'
gwasqc_sbatch=$USER_PATH/UKBB_GWAS_dev/output/select_europeans_qcbed_$(date +"%Y-%m-%d").sbatch

gwasqc1_args="""qc:1
    --cwd $cwd
    --genoFile $genoFile
    --keep_samples $keep_samples
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
    --mem $mem
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $gwasqc_sos \
    --to-script $gwasqc_sbatch \
    --args "$gwasqc1_args"

## Step 2 . Run KING

In [ ]:
cwd=$UKBB_PATH/results/083021_PCA_results/090221_king/
genoFile=$UKBB_PATH/results/083021_PCA_results/europeans/cache/UKB_genotypedatadownloaded083019.090221_sample_variant_qc_final_callrate90.filtered.extracted.europeans.filtered.bed
king_sbatch=$OUT_PATH/flashpca_king_extendedwhite_$(date +"%Y-%m-%d").sbatch
kinship=0.0625
numThreads=20
mem='30G'
walltime='36h'

king_args="""king
    --cwd $cwd
    --genoFile $genoFile
    --kinship $kinship
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
    --mem $mem
    --walltime $walltime
    --no-maximize-unrelated
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $gwasqc_sos \
    --to-script $king_sbatch \
    --args "$king_args"

## Step 3 . Remove related and LD pruning

In [ ]:
cwd=$UKBB_PATH/results/083021_PCA_results/090221_ldprun_unrelated
## Use the qc version of the genotype array with the already filtered 189010 white individuals
genoFile=$UKBB_PATH/results/083021_PCA_results/europeans/cache/*.europeans.filtered.bed
#To keep the samples of white individuals only
remove_samples=$UKBB_PATH/results/083021_PCA_results/090221_king/*.related_id

#GWAS QC variables: leave all the variables in 0 so there's no more filtering in the already filtered data
maf_filter=0
geno_filter=0
hwe_filter=0
mind_filter=0
#LD prunning variables
window=50
shift=10
r2=0.1
gwas_sbatch=$OUT_PATH/gwas_unrelated_european_$(date +"%Y-%m-%d").sbatch
numThreads=20
mem='30G'

gwasqc_args="""qc
    --cwd $cwd
    --genoFile $genoFile
    --remove_samples $remove_samples
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --window $window
    --shift $shift
    --r2 $r2
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
    --mem $mem
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg\
    --template-file $tpl_file \
    --workflow-file $gwasqc_sos \
    --to-script $gwas_sbatch \
    --args "$gwasqc_args"

## Step 4. Get bed file for related

In [ ]:
##Columbia's cluster
cwd=$UKBB_PATH/results/083021_PCA_results/090221_ldprun_related
genoFile=$UKBB_PATH/results/083021_PCA_results/europeans/cache/*.europeans.filtered.bed
keep_samples=$UKBB_PATH/results/083021_PCA_results/090221_king/*.related_id
keep_variants=$UKBB_PATH/results/083021_PCA_results/090221_ldprun_unrelated/cache/*.090221_ldprun_unrelated.filtered.prune.in

#GWAS QC variables
maf_filter=0.0
geno_filter=0.0
hwe_filter=0.0
mind_filter=0.0
gwas_sbatch=$OUT_PATH/gwas_related_european_$(date +"%Y-%m-%d").sbatch
numThreads=20
mem='30G'

gwasqc_args="""qc:1
    --cwd $cwd
    --genoFile $genoFile
    --keep_samples $keep_samples
    --keep_variants $keep_variants
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
    --mem $mem
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg\
    --template-file $tpl_file \
    --workflow-file $gwasqc_sos \
    --to-script $gwas_sbatch \
    --args "$gwasqc_args"

## Step 5. Run PCA with unrelated samples

In [ ]:
cwd=$UKBB_PATH/results/083021_PCA_results/090321_PCA_unrelated
#This is the bfile originated after filtering unrelated individuals and pruning
genoFile=$UKBB_PATH/results/083021_PCA_results/090221_ldprun_unrelated/cache/*.090221_ldprun_unrelated.filtered.prune.bed
phenoFile=$UKBB_yale/pleiotropy_R01/ukb43978_OCT2020/dc2325_phenotypes/030821_ukb42495_exomed_white_189010ind.pheno
label_col=ethnicity
pop_col=ethnicity
pca_sbatch=$OUT_PATH/flashpca_european_unrelated_$(date +"%Y-%m-%d").sbatch
k=10
maha_k=5
min_axis=0
max_axis=0

pca_args="""flashpca
    --cwd $cwd
    --genoFile $genoFile
    --phenoFile $phenoFile
    --k $k
    --maha_k $maha_k
    --label_col $label_col
    --pop_col $pop_col
    --min_axis $min_axis
    --max_axis $max_axis
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $pca_sos \
    --to-script $pca_sbatch \
    --args "$pca_args"

## Step 6. Project related samples back

https://privefl.github.io/blog/detecting-outlier-samples-in-pca/

In [ ]:
cwd=$UKBB_PATH/results/083021_PCA_results/090321_PCA_related_pval0.005
#This is the bfile originated after filtering related individuals
genoFile=$UKBB_PATH/results/083021_PCA_results/090221_ldprun_related/cache/*.filtered.extracted.bed
phenoFile=$UKBB_yale/pleiotropy_R01/ukb43978_OCT2020/dc2325_phenotypes/030821_ukb42495_exomed_white_189010ind.pheno
# Project using the PCA model for the unrelated individuals
pca_model=$UKBB_PATH/results/083021_PCA_results/090321_PCA_unrelated/*.pca.rds
pca_sbatch=$OUT_PATH/flashpca_european_related_projected_pval0.005_$(date +"%Y-%m-%d").sbatch
label_col=ethnicity
pop_col=ethnicity
k=10
maha_k=10
prob=0.997
#after correcting for multiple comparissons 0.05/10PC's
pval=0.005
min_axis=0
max_axis=0
## set the --homogeneous options to consider all the pops like one 
## For the plot you need to use the *.projected.rds and not the *.projected.mahalanobis.rds
#plot_data=$UKBB_PATH/results/070921_pca_genotype_array/white_expanded_06_30_21_genoarray_projected/030821_ukb42495_exomed_white_189010ind.pheno.white_expanded_06_30_21_genoarray_projected.pca.projected.rds
#outlier_file=$UKBB_PATH/results/070921_pca_genotype_array/white_expanded_06_30_21_genoarray_projected/030821_ukb42495_exomed_white_189010ind.pheno.white_expanded_06_30_21_genoarray_projected.pca.projected.outliers


pca_args="""project_samples
    --cwd $cwd
    --genoFile $genoFile
    --phenoFile $phenoFile
    --pca_model $pca_model
    --k $k
    --maha_k $maha_k
    --label_col $label_col
    --pop_col $pop_col
    --prob $prob
    --pval $pval
    --homogeneous
    --min_axis $min_axis
    --max_axis $max_axis
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $pca_sos \
    --to-script $pca_sbatch \
    --args "$pca_args"

## Step 7. Plot the projected individuals and highlight outliers

In [ ]:
cwd=$UKBB_PATH/results/083021_PCA_results/090321_PCA_related_pval0.005
#This is the bfile originated after filtering related individuals
genoFile=$UKBB_PATH/results/083021_PCA_results/090221_ldprun_related/cache/*.filtered.extracted.bed
phenoFile=$UKBB_yale/pleiotropy_R01/ukb43978_OCT2020/dc2325_phenotypes/030821_ukb42495_exomed_white_189010ind.pheno
plot_data=$UKBB_PATH/results/083021_PCA_results/090321_PCA_related_pval0.005/*.pca.projected.rds
outlier_file=$UKBB_PATH/results/083021_PCA_results/090321_PCA_related_pval0.005/*.outliers
label_col=ethnicity
pop_col=ethnicity
pca_sbatch=$OUT_PATH/plot_european_projected_$(date +"%Y-%m-%d").sbatch

pca_args="""plot_pca
    --cwd $cwd
    --genoFile $genoFile
    --phenoFile $phenoFile
    --label_col $label_col
    --pop_col $pop_col
    --plot_data $plot_data
    --outlier_file $outlier_file
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $pca_sos \
    --to-script $pca_sbatch \
    --args "$pca_args"

## 09-03-21 Re-run the PCA for every phenotype to obtain PC's for LMM analysis. 

### Mega sample

### H-aid

#### Step 1.

In [ ]:
cwd=$UKBB_PATH/results/083021_PCA_results/090821_f3393_pca
gwas_sbatch=$OUT_PATH/qc1_f3393_qcarray_$(date +"%Y-%m-%d").sbatch
## Use qc'ed genotype array
genoFile=~/UKBiobank/genotype_files_processed/090221_sample_variant_qc_final_callrate90/cache/UKB_genotypedatadownloaded083019.090221_sample_variant_qc_final_callrate90.filtered.extracted.bed
keep_samples=$UKBB_PATH/results/083021_PCA_results/090321_UKBB_Hearing_aid_f3393_expandedwhite_6436cases_96601ctrl.keep_id
#Keep variants after LD pruning
keep_variants=$UKBB_PATH/results/083021_PCA_results/090221_ldprun_unrelated/cache/*.090221_ldprun_unrelated.filtered.prune.in
#GWAS QC variables set all of this variables to 0 to avoid doing more filtering
maf_filter=0
geno_filter=0
hwe_filter=0
mind_filter=0
mem='30G'

gwasqc_args="""qc:1
    --cwd $cwd
    --genoFile $genoFile
    --keep_samples $keep_samples
    --keep_variants $keep_variants
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
    --mem $mem
"""

sos run $USER_PATH/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $gwasqc_sos \
    --to-script $gwas_sbatch \
    --args "$gwasqc_args"

#### Step 2. 

In [ ]:
cwd=$UKBB_PATH/results/083021_PCA_results/090821_f3393_pca
#This is the bfile obtained in step 1
genoFile=$UKBB_PATH/results/083021_PCA_results/090821_f3393_pca/cache/*.bed
# Format FID, IID, pop
phenoFile=$UKBB_PATH/results/083021_PCA_results/090321_UKBB_Hearing_aid_f3393_expandedwhite_6436cases_96601ctrl.phenopca
label_col=ethnicity
pop_col=ethnicity
pca_sbatch=$OUT_PATH/flashpca_f3393_pc_$(date +"%Y-%m-%d").sbatch
k=2
min_axis=0
max_axis=0

pca_args="""flashpca
    --cwd $cwd
    --genoFile $genoFile
    --phenoFile $phenoFile
    --k $k
    --label_col $label_col
    --pop_col $pop_col
    --min_axis $min_axis
    --max_axis $max_axis
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
"""

sos run  $USER_PATH/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg\
    --template-file $tpl_file \
    --workflow-file $pca_sos \
    --to-script $pca_sbatch \
    --args "$pca_args"

### H-noise

#### Step 1.

In [ ]:
cwd=$UKBB_PATH/results/083021_PCA_results/090821_f2247_pca
gwas_sbatch=$OUT_PATH/qc1_f2247_qcarray_$(date +"%Y-%m-%d").sbatch
## Use qc'ed genotype array
genoFile=~/UKBiobank/genotype_files_processed/090221_sample_variant_qc_final_callrate90/cache/UKB_genotypedatadownloaded083019.090221_sample_variant_qc_final_callrate90.filtered.extracted.bed
keep_samples=$UKBB_PATH/results/083021_PCA_results/090321_UKBB_Hearing_difficulty_f2247_expandedwhite_45502cases_96601ctrl.keep_id
keep_variants=$UKBB_PATH/results/083021_PCA_results/090221_ldprun_unrelated/cache/*.090221_ldprun_unrelated.filtered.prune.in
#GWAS QC variables set all of this variables to 0 to avoid doing more filtering
maf_filter=0
geno_filter=0
hwe_filter=0
mind_filter=0
mem='30G'

gwasqc_args="""qc:1
    --cwd $cwd
    --genoFile $genoFile
    --keep_samples $keep_samples
    --keep_variants $keep_variants
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
    --mem $mem
"""

sos run $USER_PATH/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $gwasqc_sos \
    --to-script $gwas_sbatch \
    --args "$gwasqc_args"

#### Step 2.

In [ ]:
cwd=$UKBB_PATH/results/083021_PCA_results/090821_f2247_pca
#This is the bfile obtained in step 1
genoFile=$UKBB_PATH/results/083021_PCA_results/090821_f2247_pca/cache/*.bed
# Format FID, IID, pop
phenoFile=$UKBB_PATH/results/083021_PCA_results/090321_UKBB_Hearing_difficulty_f2247_expandedwhite_45502cases_96601ctrl.phenopca
label_col=ethnicity
pop_col=ethnicity
pca_sbatch=$OUT_PATH/flashpca_f2247_pc_$(date +"%Y-%m-%d").sbatch
k=2
min_axis=0
max_axis=0

pca_args="""flashpca
    --cwd $cwd
    --genoFile $genoFile
    --phenoFile $phenoFile
    --k $k
    --label_col $label_col
    --pop_col $pop_col
    --min_axis $min_axis
    --max_axis $max_axis
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
"""

sos run  $USER_PATH/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg\
    --template-file $tpl_file \
    --workflow-file $pca_sos \
    --to-script $pca_sbatch \
    --args "$pca_args"

### H-diff

#### Step 1.

In [ ]:
cwd=$UKBB_PATH/results/083021_PCA_results/090821_f2257_pca
gwas_sbatch=$OUT_PATH/qc1_f2257_qcarray_$(date +"%Y-%m-%d").sbatch
## Use qc'ed genotype array
genoFile=~/UKBiobank/genotype_files_processed/090221_sample_variant_qc_final_callrate90/cache/UKB_genotypedatadownloaded083019.090221_sample_variant_qc_final_callrate90.filtered.extracted.bed
keep_samples=$UKBB_PATH/results/083021_PCA_results/090321_UKBB_Hearing_noise_f2257_expandedwhite_65660cases_96601ctrl.keep_id
keep_variants=$UKBB_PATH/results/083021_PCA_results/090221_ldprun_unrelated/cache/*.090221_ldprun_unrelated.filtered.prune.in
#GWAS QC variables set all of this variables to 0 to avoid doing more filtering
maf_filter=0
geno_filter=0
hwe_filter=0
mind_filter=0
mem='30G'

gwasqc_args="""qc:1
    --cwd $cwd
    --genoFile $genoFile
    --keep_samples $keep_samples
    --keep_variants $keep_variants
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
    --mem $mem
"""

sos run $USER_PATH/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $gwasqc_sos \
    --to-script $gwas_sbatch \
    --args "$gwasqc_args"

#### Step 2.

In [ ]:
cwd=$UKBB_PATH/results/083021_PCA_results/090821_f2257_pca
#This is the bfile obtained in step 1
genoFile=$UKBB_PATH/results/083021_PCA_results/090821_f2257_pca/cache/*.bed
# Format FID, IID, pop
phenoFile=$UKBB_PATH/results/083021_PCA_results/090321_UKBB_Hearing_noise_f2257_expandedwhite_65660cases_96601ctrl.phenopca
label_col=ethnicity
pop_col=ethnicity
pca_sbatch=$OUT_PATH/flashpca_f2257_pc_$(date +"%Y-%m-%d").sbatch
k=2
min_axis=0
max_axis=0

pca_args="""flashpca
    --cwd $cwd
    --genoFile $genoFile
    --phenoFile $phenoFile
    --k $k
    --label_col $label_col
    --pop_col $pop_col
    --min_axis $min_axis
    --max_axis $max_axis
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
"""

sos run  $USER_PATH/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg\
    --template-file $tpl_file \
    --workflow-file $pca_sos \
    --to-script $pca_sbatch \
    --args "$pca_args"

### H-both

#### Step 1.

In [ ]:
cwd=$UKBB_PATH/results/083021_PCA_results/090821_combined_f2247_f2257_pca
gwas_sbatch=$OUT_PATH/qc1_combined_qcarray_$(date +"%Y-%m-%d").sbatch
## Use qc'ed genotype array
genoFile=~/UKBiobank/genotype_files_processed/090221_sample_variant_qc_final_callrate90/cache/UKB_genotypedatadownloaded083019.090221_sample_variant_qc_final_callrate90.filtered.extracted.bed
keep_samples=$UKBB_PATH/results/083021_PCA_results/090321_UKBB_Combined_f2247_f2257_expandedwhite_38410cases_96601ctrl.keep_id
keep_variants=$UKBB_PATH/results/083021_PCA_results/090221_ldprun_unrelated/cache/*.090221_ldprun_unrelated.filtered.prune.in
#GWAS QC variables set all of this variables to 0 to avoid doing more filtering
maf_filter=0
geno_filter=0
hwe_filter=0
mind_filter=0
mem='30G'

gwasqc_args="""qc:1
    --cwd $cwd
    --genoFile $genoFile
    --keep_samples $keep_samples
    --keep_variants $keep_variants
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
    --mem $mem
"""

sos run $USER_PATH/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $gwasqc_sos \
    --to-script $gwas_sbatch \
    --args "$gwasqc_args"

#### Step 2.

In [ ]:
cwd=$UKBB_PATH/results/083021_PCA_results/090821_combined_f2247_f2257_pca
#This is the bfile obtained in step 1
genoFile=$UKBB_PATH/results/083021_PCA_results/090821_combined_f2247_f2257_pca/cache/*.bed
# Format FID, IID, pop
phenoFile=$UKBB_PATH/results/083021_PCA_results/090321_UKBB_Combined_f2247_f2257_expandedwhite_38410cases_96601ctrl.phenopca
label_col=ethnicity
pop_col=ethnicity
pca_sbatch=$OUT_PATH/flashpca_combined_pc_$(date +"%Y-%m-%d").sbatch
k=2
min_axis=0
max_axis=0

pca_args="""flashpca
    --cwd $cwd
    --genoFile $genoFile
    --phenoFile $phenoFile
    --k $k
    --label_col $label_col
    --pop_col $pop_col
    --min_axis $min_axis
    --max_axis $max_axis
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
"""

sos run  $USER_PATH/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg\
    --template-file $tpl_file \
    --workflow-file $pca_sos \
    --to-script $pca_sbatch \
    --args "$pca_args"

## 09-14-21 Get PCA for every phenotype for the discovery and the replication samples to obtain PC's for LMM analysis. 

### H-aid Replication

#### Step 1.

In [ ]:
cwd=$UKBB_PATH/results/083021_PCA_results/091421_f3393_50Kexomes_pca
gwas_sbatch=$OUT_PATH/qc1_f3393_qcarray_50K_$(date +"%Y-%m-%d").sbatch
## Use qc'ed genotype array
genoFile=~/UKBiobank/genotype_files_processed/090221_sample_variant_qc_final_callrate90/cache/UKB_genotypedatadownloaded083019.090221_sample_variant_qc_final_callrate90.filtered.extracted.bed
keep_samples=$UKBB_PATH/results/083021_PCA_results/090321_UKBB_Hearing_aid_f3393_expandedwhite_24189ind_50Kexomes.keep_id
#Keep variants after LD pruning
keep_variants=$UKBB_PATH/results/083021_PCA_results/090221_ldprun_unrelated/cache/*.090221_ldprun_unrelated.filtered.prune.in
#GWAS QC variables set all of this variables to 0 to avoid doing more filtering
maf_filter=0
geno_filter=0
hwe_filter=0
mind_filter=0
mem='30G'

gwasqc_args="""qc:1
    --cwd $cwd
    --genoFile $genoFile
    --keep_samples $keep_samples
    --keep_variants $keep_variants
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
    --mem $mem
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $gwasqc_sos \
    --to-script $gwas_sbatch \
    --args "$gwasqc_args"

#### Step 2

In [ ]:
cwd=$UKBB_PATH/results/083021_PCA_results/091421_f3393_50Kexomes_pca
#This is the bfile obtained in step 1
genoFile=$UKBB_PATH/results/083021_PCA_results/091421_f3393_50Kexomes_pca/cache/*.bed
# Format FID, IID, ethnicity
phenoFile=$UKBB_PATH/results/083021_PCA_results/090321_UKBB_Hearing_aid_f3393_expandedwhite_24189_50K.phenopca
label_col=ethnicity
pop_col=ethnicity
pca_sbatch=$OUT_PATH/flashpca_f3393_pc_50K_$(date +"%Y-%m-%d").sbatch
k=2
min_axis=0
max_axis=0

pca_args="""flashpca
    --cwd $cwd
    --genoFile $genoFile
    --phenoFile $phenoFile
    --k $k
    --label_col $label_col
    --pop_col $pop_col
    --min_axis $min_axis
    --max_axis $max_axis
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
"""

sos run  $USER_PATH/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg\
    --template-file $tpl_file \
    --workflow-file $pca_sos \
    --to-script $pca_sbatch \
    --args "$pca_args"

### H-aid Discovery

#### Step 1

In [ ]:
cwd=$UKBB_PATH/results/083021_PCA_results/091421_f3393_150Kexomes_pca
gwas_sbatch=$OUT_PATH/qc1_f3393_qcarray_150K_$(date +"%Y-%m-%d").sbatch
## Use qc'ed genotype array
genoFile=~/UKBiobank/genotype_files_processed/090221_sample_variant_qc_final_callrate90/cache/UKB_genotypedatadownloaded083019.090221_sample_variant_qc_final_callrate90.filtered.extracted.bed
keep_samples=$UKBB_PATH/results/083021_PCA_results/090321_UKBB_Hearing_aid_f3393_expandedwhite_78848ind_150Kexomes.keep_id
#Keep variants after LD pruning
keep_variants=$UKBB_PATH/results/083021_PCA_results/090221_ldprun_unrelated/cache/*.090221_ldprun_unrelated.filtered.prune.in
#GWAS QC variables set all of this variables to 0 to avoid doing more filtering
maf_filter=0
geno_filter=0
hwe_filter=0
mind_filter=0
mem='30G'

gwasqc_args="""qc:1
    --cwd $cwd
    --genoFile $genoFile
    --keep_samples $keep_samples
    --keep_variants $keep_variants
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
    --mem $mem
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $gwasqc_sos \
    --to-script $gwas_sbatch \
    --args "$gwasqc_args"

#### Step 2

In [ ]:
cwd=$UKBB_PATH/results/083021_PCA_results/091421_f3393_150Kexomes_pca
#This is the bfile obtained in step 1
genoFile=$UKBB_PATH/results/083021_PCA_results/091421_f3393_150Kexomes_pca/cache/*.bed
# Format FID, IID, ethnicity
phenoFile=$UKBB_PATH/results/083021_PCA_results/090321_UKBB_Hearing_aid_f3393_expandedwhite_78848_150K.phenopca
label_col=ethnicity
pop_col=ethnicity
pca_sbatch=$OUT_PATH/flashpca_f3393_pc_150K_$(date +"%Y-%m-%d").sbatch
k=2
min_axis=0
max_axis=0

pca_args="""flashpca
    --cwd $cwd
    --genoFile $genoFile
    --phenoFile $phenoFile
    --k $k
    --label_col $label_col
    --pop_col $pop_col
    --min_axis $min_axis
    --max_axis $max_axis
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
"""

sos run  $USER_PATH/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg\
    --template-file $tpl_file \
    --workflow-file $pca_sos \
    --to-script $pca_sbatch \
    --args "$pca_args"

## H-diff Replication

#### Step 1

In [ ]:
cwd=$UKBB_PATH/results/083021_PCA_results/091421_f2247_50Kexomes_pca
gwas_sbatch=$OUT_PATH/qc1_f2247_qcarray_50K_$(date +"%Y-%m-%d").sbatch
## Use qc'ed genotype array
genoFile=~/UKBiobank/genotype_files_processed/090221_sample_variant_qc_final_callrate90/cache/UKB_genotypedatadownloaded083019.090221_sample_variant_qc_final_callrate90.filtered.extracted.bed
keep_samples=$UKBB_PATH/results/083021_PCA_results/090321_UKBB_Hearing_difficulty_f2247_expandedwhite_34596ind_50Kexomes.keep_id
keep_variants=$UKBB_PATH/results/083021_PCA_results/090221_ldprun_unrelated/cache/*.090221_ldprun_unrelated.filtered.prune.in
#GWAS QC variables set all of this variables to 0 to avoid doing more filtering
maf_filter=0
geno_filter=0
hwe_filter=0
mind_filter=0
numThreads=1
mem='30G'

gwasqc_args="""qc:1
    --cwd $cwd
    --genoFile $genoFile
    --keep_samples $keep_samples
    --keep_variants $keep_variants
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
    --mem $mem
"""

sos run $USER_PATH/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $gwasqc_sos \
    --to-script $gwas_sbatch \
    --args "$gwasqc_args"

#### Step 2.

In [ ]:
cwd=$UKBB_PATH/results/083021_PCA_results/091421_f2247_50Kexomes_pca
#This is the bfile obtained in step 1
genoFile=$UKBB_PATH/results/083021_PCA_results/091421_f2247_50Kexomes_pca/cache/*.bed
# Format FID, IID, ethnicity
phenoFile=$UKBB_PATH/results/083021_PCA_results/090321_UKBB_Hearing_difficulty_f2247_expandedwhite_34596ind_50K.phenopca
label_col=ethnicity
pop_col=ethnicity
pca_sbatch=$OUT_PATH/flashpca_f2247_pc_50K_$(date +"%Y-%m-%d").sbatch
k=2
min_axis=0
max_axis=0

pca_args="""flashpca
    --cwd $cwd
    --genoFile $genoFile
    --phenoFile $phenoFile
    --k $k
    --label_col $label_col
    --pop_col $pop_col
    --min_axis $min_axis
    --max_axis $max_axis
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
"""

sos run  $USER_PATH/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg\
    --template-file $tpl_file \
    --workflow-file $pca_sos \
    --to-script $pca_sbatch \
    --args "$pca_args"

## H-diff Discovery

#### Step 1. 

In [ ]:
cwd=$UKBB_PATH/results/083021_PCA_results/091421_f2247_150Kexomes_pca
gwas_sbatch=$OUT_PATH/qc1_f2247_qcarray_150K_$(date +"%Y-%m-%d").sbatch
## Use qc'ed genotype array
genoFile=~/UKBiobank/genotype_files_processed/090221_sample_variant_qc_final_callrate90/cache/UKB_genotypedatadownloaded083019.090221_sample_variant_qc_final_callrate90.filtered.extracted.bed
keep_samples=$UKBB_PATH/results/083021_PCA_results/090321_UKBB_Hearing_difficulty_f2247_expandedwhite_107507ind_150Kexomes.keep_id
keep_variants=$UKBB_PATH/results/083021_PCA_results/090221_ldprun_unrelated/cache/*.090221_ldprun_unrelated.filtered.prune.in
#GWAS QC variables set all of this variables to 0 to avoid doing more filtering
maf_filter=0
geno_filter=0
hwe_filter=0
mind_filter=0
mem='30G'

gwasqc_args="""qc:1
    --cwd $cwd
    --genoFile $genoFile
    --keep_samples $keep_samples
    --keep_variants $keep_variants
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
    --mem $mem
"""

sos run $USER_PATH/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $gwasqc_sos \
    --to-script $gwas_sbatch \
    --args "$gwasqc_args"

#### Step 2.

In [ ]:
cwd=$UKBB_PATH/results/083021_PCA_results/091421_f2247_150Kexomes_pca
#This is the bfile obtained in step 1
genoFile=$UKBB_PATH/results/083021_PCA_results/091421_f2247_150Kexomes_pca/cache/*.bed
# Format FID, IID, ethnicity
phenoFile=$UKBB_PATH/results/083021_PCA_results/090321_UKBB_Hearing_difficulty_f2247_expandedwhite_107507ind_150K.phenopca
label_col=ethnicity
pop_col=ethnicity
pca_sbatch=$OUT_PATH/flashpca_f2247_pc_150K_$(date +"%Y-%m-%d").sbatch
k=2
min_axis=0
max_axis=0

pca_args="""flashpca
    --cwd $cwd
    --genoFile $genoFile
    --phenoFile $phenoFile
    --k $k
    --label_col $label_col
    --pop_col $pop_col
    --min_axis $min_axis
    --max_axis $max_axis
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
"""

sos run  $USER_PATH/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg\
    --template-file $tpl_file \
    --workflow-file $pca_sos \
    --to-script $pca_sbatch \
    --args "$pca_args"

## H-noise Replication

#### Step 1

In [ ]:
cwd=$UKBB_PATH/results/083021_PCA_results/091421_f2257_50Kexomes_pca
gwas_sbatch=$OUT_PATH/qc1_f2257_qcarray_50K_$(date +"%Y-%m-%d").sbatch
## Use qc'ed genotype array
genoFile=~/UKBiobank/genotype_files_processed/090221_sample_variant_qc_final_callrate90/cache/UKB_genotypedatadownloaded083019.090221_sample_variant_qc_final_callrate90.filtered.extracted.bed
keep_samples=$UKBB_PATH/results/083021_PCA_results/090321_UKBB_Hearing_noise_f2257_expandedwhite_38721ind_50Kexomes.keep_id
keep_variants=$UKBB_PATH/results/083021_PCA_results/090221_ldprun_unrelated/cache/*.090221_ldprun_unrelated.filtered.prune.in
#GWAS QC variables set all of this variables to 0 to avoid doing more filtering
maf_filter=0
geno_filter=0
hwe_filter=0
mind_filter=0
mem='30G'

gwasqc_args="""qc:1
    --cwd $cwd
    --genoFile $genoFile
    --keep_samples $keep_samples
    --keep_variants $keep_variants
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
    --mem $mem
"""

sos run $USER_PATH/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $gwasqc_sos \
    --to-script $gwas_sbatch \
    --args "$gwasqc_args"

#### Step 2

In [ ]:
cwd=$UKBB_PATH/results/083021_PCA_results/091421_f2257_50Kexomes_pca
#This is the bfile obtained in step 1
genoFile=$UKBB_PATH/results/083021_PCA_results/091421_f2257_50Kexomes_pca/cache/*.bed
# Format FID, IID, ethnicity
phenoFile=$UKBB_PATH/results/083021_PCA_results/090321_UKBB_Hearing_noise_f2257_expandedwhite_38723ind_50K.phenopca
label_col=ethnicity
pop_col=ethnicity
pca_sbatch=$OUT_PATH/flashpca_f2257_pc_50K_$(date +"%Y-%m-%d").sbatch
k=2
min_axis=0
max_axis=0

pca_args="""flashpca
    --cwd $cwd
    --genoFile $genoFile
    --phenoFile $phenoFile
    --k $k
    --label_col $label_col
    --pop_col $pop_col
    --min_axis $min_axis
    --max_axis $max_axis
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
"""

sos run  $USER_PATH/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg\
    --template-file $tpl_file \
    --workflow-file $pca_sos \
    --to-script $pca_sbatch \
    --args "$pca_args"

## H-diff Discovery

#### Step 1.

In [ ]:
cwd=$UKBB_PATH/results/083021_PCA_results/091421_f2257_150Kexomes_pca
gwas_sbatch=$OUT_PATH/qc1_f2257_qcarray_150K_$(date +"%Y-%m-%d").sbatch
## Use qc'ed genotype array
genoFile=~/UKBiobank/genotype_files_processed/090221_sample_variant_qc_final_callrate90/cache/UKB_genotypedatadownloaded083019.090221_sample_variant_qc_final_callrate90.filtered.extracted.bed
keep_samples=$UKBB_PATH/results/083021_PCA_results/090321_UKBB_Hearing_noise_f2257_expandedwhite_123538ind_150Kexomes.keep_id
keep_variants=$UKBB_PATH/results/083021_PCA_results/090221_ldprun_unrelated/cache/*.090221_ldprun_unrelated.filtered.prune.in
#GWAS QC variables set all of this variables to 0 to avoid doing more filtering
maf_filter=0
geno_filter=0
hwe_filter=0
mind_filter=0
numThreads=1
mem='30G'

gwasqc_args="""qc:1
    --cwd $cwd
    --genoFile $genoFile
    --keep_samples $keep_samples
    --keep_variants $keep_variants
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
    --mem $mem
"""

sos run $USER_PATH/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $gwasqc_sos \
    --to-script $gwas_sbatch \
    --args "$gwasqc_args"

#### Step 2. 

In [ ]:
cwd=$UKBB_PATH/results/083021_PCA_results/091421_f2257_150Kexomes_pca
#This is the bfile obtained in step 1
genoFile=$UKBB_PATH/results/083021_PCA_results/091421_f2257_150Kexomes_pca/cache/*.bed
# Format FID, IID, ethnicity
phenoFile=$UKBB_PATH/results/083021_PCA_results/090321_UKBB_Hearing_noise_f2257_expandedwhite_123538ind_150K.phenopca
label_col=ethnicity
pop_col=ethnicity
pca_sbatch=$OUT_PATH/flashpca_f2257_pc_150K_$(date +"%Y-%m-%d").sbatch
k=2
min_axis=0
max_axis=0

pca_args="""flashpca
    --cwd $cwd
    --genoFile $genoFile
    --phenoFile $phenoFile
    --k $k
    --label_col $label_col
    --pop_col $pop_col
    --min_axis $min_axis
    --max_axis $max_axis
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
"""

sos run  $USER_PATH/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg\
    --template-file $tpl_file \
    --workflow-file $pca_sos \
    --to-script $pca_sbatch \
    --args "$pca_args"

## H-both Replication

#### Step 1

In [ ]:
cwd=$UKBB_PATH/results/083021_PCA_results/091421_combined_f2247_f2257_50Kexomes_pca
gwas_sbatch=$OUT_PATH/qc1_combined_qcarray_50K_$(date +"%Y-%m-%d").sbatch
## Use qc'ed genotype array
genoFile=~/UKBiobank/genotype_files_processed/090221_sample_variant_qc_final_callrate90/cache/UKB_genotypedatadownloaded083019.090221_sample_variant_qc_final_callrate90.filtered.extracted.bed
keep_samples=$UKBB_PATH/results/083021_PCA_results/090321_UKBB_Combined_f2247_f2257_expandedwhite_32878ind_50Kexomes.keep_id
keep_variants=$UKBB_PATH/results/083021_PCA_results/090221_ldprun_unrelated/cache/*.090221_ldprun_unrelated.filtered.prune.in
#GWAS QC variables set all of this variables to 0 to avoid doing more filtering
maf_filter=0
geno_filter=0
hwe_filter=0
mind_filter=0
mem='30G'

gwasqc_args="""qc:1
    --cwd $cwd
    --genoFile $genoFile
    --keep_samples $keep_samples
    --keep_variants $keep_variants
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
    --mem $mem
"""

sos run $USER_PATH/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $gwasqc_sos \
    --to-script $gwas_sbatch \
    --args "$gwasqc_args"

#### Step 2. 

In [ ]:
cwd=$UKBB_PATH/results/083021_PCA_results/091421_combined_f2247_f2257_50Kexomes_pca
#This is the bfile obtained in step 1
genoFile=$UKBB_PATH/results/083021_PCA_results/091421_combined_f2247_f2257_50Kexomes_pca/cache/*.bed
# Format FID, IID, ethnicity
phenoFile=$UKBB_PATH/results/083021_PCA_results/090321_UKBB_Combined_f2247_f2257_expandedwhite_32878ind_50K.phenopca
label_col=ethnicity
pop_col=ethnicity
pca_sbatch=$OUT_PATH/flashpca_combined_pc_50K_$(date +"%Y-%m-%d").sbatch
k=2
min_axis=0
max_axis=0

pca_args="""flashpca
    --cwd $cwd
    --genoFile $genoFile
    --phenoFile $phenoFile
    --k $k
    --label_col $label_col
    --pop_col $pop_col
    --min_axis $min_axis
    --max_axis $max_axis
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
"""

sos run  $USER_PATH/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg\
    --template-file $tpl_file \
    --workflow-file $pca_sos \
    --to-script $pca_sbatch \
    --args "$pca_args"

## H-both Discovery

#### Step 1.

In [ ]:
cwd=$UKBB_PATH/results/083021_PCA_results/091421_combined_f2247_f2257_150Kexomes_pca
gwas_sbatch=$OUT_PATH/qc1_combined_qcarray_150K_$(date +"%Y-%m-%d").sbatch
## Use qc'ed genotype array
genoFile=~/UKBiobank/genotype_files_processed/090221_sample_variant_qc_final_callrate90/cache/UKB_genotypedatadownloaded083019.090221_sample_variant_qc_final_callrate90.filtered.extracted.bed
keep_samples=$UKBB_PATH/results/083021_PCA_results/090321_UKBB_Combined_f2247_f2257_expandedwhite_102133ind_150Kexomes.keep_id
keep_variants=$UKBB_PATH/results/083021_PCA_results/090221_ldprun_unrelated/cache/*.090221_ldprun_unrelated.filtered.prune.in
#GWAS QC variables set all of this variables to 0 to avoid doing more filtering
maf_filter=0
geno_filter=0
hwe_filter=0
mind_filter=0
mem='30G'

gwasqc_args="""qc:1
    --cwd $cwd
    --genoFile $genoFile
    --keep_samples $keep_samples
    --keep_variants $keep_variants
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
    --mem $mem
"""

sos run $USER_PATH/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $gwasqc_sos \
    --to-script $gwas_sbatch \
    --args "$gwasqc_args"

#### Step 2.

In [ ]:
cwd=$UKBB_PATH/results/083021_PCA_results/091421_combined_f2247_f2257_150Kexomes_pca
#This is the bfile obtained in step 1
genoFile=$UKBB_PATH/results/083021_PCA_results/091421_combined_f2247_f2257_150Kexomes_pca/cache/*.bed
# Format FID, IID, ethnicity
phenoFile=$UKBB_PATH/results/083021_PCA_results/090321_UKBB_Combined_f2247_f2257_expandedwhite_102133ind_150K.phenopca
label_col=ethnicity
pop_col=ethnicity
pca_sbatch=$OUT_PATH/flashpca_combined_pc_150K_$(date +"%Y-%m-%d").sbatch
k=2
min_axis=0
max_axis=0

pca_args="""flashpca
    --cwd $cwd
    --genoFile $genoFile
    --phenoFile $phenoFile
    --k $k
    --label_col $label_col
    --pop_col $pop_col
    --min_axis $min_axis
    --max_axis $max_axis
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
"""

sos run  $USER_PATH/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg\
    --template-file $tpl_file \
    --workflow-file $pca_sos \
    --to-script $pca_sbatch \
    --args "$pca_args"